## Basic setup

In [10]:
%run env_setup.py
%matplotlib inline
from __future__ import division,print_function
from importlib import reload
import numpy as np
import os
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

## Download and setup

**you can change the data and the batch size to fit the model to other datasets**

In [5]:
# Download
from lessdeep.utils.datasets.kaggle import dog_cat
datapath = dog_cat.download_data()

In [4]:
## As large as you can, but no larger than 64 is recommanded
batch_size = 64

## Create model

In [11]:
# Generate vgg model
import lessdeep.model.vgg16
reload(lessdeep.model.vgg16)
from lessdeep.model.vgg16 import Vgg16

vgg = Vgg16()

## Load data(Batches)
- accept data path contain at least two folder
    - train
    - valid
- each subfolder contain images in each folder with the class name

In [12]:
# depends on the variable datapath
batches = vgg.get_batches(os.path.join(datapath, 'train'), batch_size=batch_size)
val_batches = vgg.get_batches(os.path.join(datapath, 'valid'), batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Finetune

## Test on sample

## Start train

## Bad result

## Good result